In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
# Reconnect to the database
conn = sqlite3.connect('./ipl_database.db')
# Query the database
df_ball = pd.read_sql_query('SELECT * FROM ipl_ball_by_ball;', conn)

# Display the first few rows of the table
print(df_ball)

# Close the connection
conn.close()

In [ ]:
vk_df = df_ball[df_ball['batter'] == 'V Kohli']
print(len(vk_df))
vk_df

In [ ]:
total_runs = vk_df['batsman_run'].sum()

print("Total runs scored by V Kohli:", total_runs)

In [ ]:
unique_batters = df_ball['batter'].unique().tolist()

unique_batters

In [ ]:
batter_dict = {}
for i in unique_batters:
    df_batter = df_ball[df_ball['batter'] == i]
    total_runs = df_batter['batsman_run'].sum()
    batter_dict[i] = total_runs

In [ ]:
batter_dict

In [ ]:
unique_bowlers = df_ball['bowler'].unique().tolist()

unique_bowlers

In [ ]:
df_ball['isWicketDelivery'] = pd.to_numeric(df_ball['isWicketDelivery'], errors='coerce').fillna(0).astype(int)

# Filter for deliveries by R Ashwin that resulted in a wicket
ashwin_wickets = df_ball[(df_ball['bowler'] == 'R Ashwin') & (df_ball['isWicketDelivery'] == 1) & (df_ball['kind'] != "run out")]

wicket_count = ashwin_wickets.shape[0]

print("Wickets taken by R Ashwin:", wicket_count)
ashwin_wickets

In [ ]:
bowler_dict = {}
for i in unique_bowlers:
    df_ball['isWicketDelivery'] = pd.to_numeric(df_ball['isWicketDelivery'], errors='coerce').fillna(0).astype(int)
    bowler_wickets = df_ball[(df_ball['bowler'] == i) & (df_ball['isWicketDelivery'] == 1) & (df_ball['kind'] != "run out")]
    wicket_count = bowler_wickets.shape[0]
    print("Wickets taken by ",i,":", wicket_count)
    
    condition = (df_ball['bowler'] == i) & (df_ball['extra_type'] != "legbyes")

    bowling_deliveries = df_ball[condition]

    total_runs = bowling_deliveries['total_run'].sum()

    print("Total runs conceded by ",i,":", total_runs)

    bowler_legal_deliveries = df_ball[(df_ball['bowler'] == i) & (df_ball['extra_type'] != "wides") & (df_ball['extra_type'] != "noballs")]
    bowler_legal_deliveries_count = bowler_legal_deliveries.shape[0]

    print("Total deliveries bowled by ",i,":", bowler_legal_deliveries_count)
    

    bowl_deliveries = df_ball[df_ball['bowler'] == i]

    grouped = bowl_deliveries.groupby(['ID', 'overs'])['ballnumber'].apply(list)

    # Construct the dictionary
    bowl_overs_dict = {}
    for (match_id, over), balls in grouped.items():
        if match_id not in bowl_overs_dict:
            bowl_overs_dict[match_id] = []
        bowl_overs_dict[match_id].append((over, balls))

    # Display the dictionary
    for match_id, overs in bowl_overs_dict.items():
        print(f"Match ID {match_id}: {overs}")


    bowling_avg = 0
    if wicket_count == 0:
        bowling_avg = np.nan
    else:
        bowling_avg = total_runs/wicket_count
    print("Bowling average of ",i,":", bowling_avg)
    bowler_dict[i] = [total_runs,bowler_legal_deliveries_count,wicket_count,bowling_avg,bowl_overs_dict]

In [ ]:
bowler_dict